In [2]:
import pandas as pd
import numpy as np
import os
import shutil
from pathlib import Path
import nltk
import string
import json
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import pickle
import math

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
f = open("id_to_tokens.txt","rb")
txt = f.read()
f.close()
id_to_tokens = pickle.loads(txt)

In [8]:
f = open("id_to_name.txt","rb")
txt = f.read()
f.close()
id_to_name = pickle.loads(txt)

In [7]:
f = open("name_to_id.txt","rb")
txt = f.read()
f.close()
name_to_id = pickle.loads(txt)

In [6]:
def process_query(query):
    query = query.lower()
    query = query.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(query)
    tokens_without_sw = [word for word in tokens if not word in stopwords.words('english')]
    return tokens_without_sw

In [5]:
def jaccard_single_doc(query_token_list,document_token_list):
    query_set = set(query_token_list)
    document_set = set(document_token_list)
    intersected_list = list(query_set & document_set)
    union_list = list(query_set | document_set)
    jacc_coeff = len(intersected_list)/len(union_list)
    return jacc_coeff


In [4]:
def relevant_by_jaccard(query,top_n_docs):
    query_token_list = process_query(query)
    total_docs = len(id_to_tokens)
    id_to_jaccard = {}
    for i in id_to_tokens:
        document_token_list = id_to_tokens[i]
        jacc_coeff = jaccard_single_doc(query_token_list,document_token_list)
        id_to_jaccard[i] = jacc_coeff

    id_to_jaccard = dict(sorted(id_to_jaccard.items(), key=lambda item: item[1],reverse=True))
    top_ids = list(id_to_jaccard.keys())
    for i in range(top_n_docs):
        print(id_to_name[top_ids[i]])
        # print(id_to_jaccard[top_ids[i]])


In [10]:
relevant_by_jaccard("Copyrighted November 28, 1992, by Curtis H. Hoffmann",5)

flowchrt.txt
jambalay.pol
oxymoron.txt
pbcookie.des
temphell.jok


In [ ]:
#Uncomment if want to build fresh vocab

# vocab = []
# vocab = set(vocab)

# for i in id_to_tokens:
#     tokens = id_to_tokens[i]
#     tokens = set(tokens)
#     vocab = vocab | tokens

# vocab = list(vocab)
# len(vocab)

71038

In [ ]:
# Uncomment all below if want to build fresh weight matrix


#unique_terms_plus_doc_freq = {}

# for i in id_to_tokens:
#     tokens_of_i = id_to_tokens[i]
#     for j in tokens_of_i:
#         if j in unique_terms_plus_doc_freq:
#             unique_terms_plus_doc_freq[j]+=1
#         else:
#             unique_terms_plus_doc_freq[j]=1
        
# # print(len(unique_terms_plus_doc_freq))

71038


In [ ]:
# document_term_mat = {}

# for i in id_to_tokens:
#     nested_dictionary = {}
#     all_tokens = id_to_tokens[i]
#     for j in all_tokens:
#         if (j in nested_dictionary):
#             nested_dictionary[j]+=1
#         else:
#             nested_dictionary[j]=1
#     document_term_mat[i] = nested_dictionary

In [ ]:
# def idf(term):
#     total_docs = len(id_to_tokens)
#     doc_frequency = unique_terms_plus_doc_freq[term]
#     division = (total_docs/doc_frequency)
#     division += 1
#     idf_term = math.log10(division)
#     return idf_term

In [ ]:
# def tf(term, document_id):
#     tf_td = 0
#     if term not in document_term_mat[document_id]:
#         tf_td = 0
#     else:
#         tf_td = document_term_mat[document_id][term]

#     document_all_term_freq = document_term_mat[document_id]
#     total = 0
#     for i in document_all_term_freq:
#         total += document_all_term_freq[i]

#     term_freq_values = document_all_term_freq.values()
#     max_term_freq = max(term_freq_values)
    
#     tf_binary = 0
#     if (tf_td != 0):
#         tf_binary = 1

#     tf_rawcount = tf_td

#     tf_termfrequency = (tf_td/total)

#     tf_lognorm = math.log10(1+tf_td)

#     tf_doublenorm = 0.5 + 0.5*(tf_td/max_term_freq)

#     return [tf_binary,tf_rawcount,tf_termfrequency,tf_lognorm,tf_doublenorm]



In [ ]:
# def tf_idf(term,document_id):
#     idf_t = idf(term)
#     tf_td = tf(term,document_id)
#     tf_idf_td = []
#     for i in tf_td:
#         tf_idf_td.append(i*idf_t)
#     return tf_idf_td

In [1]:
#uncomment if want to construct fresh weight matrix


#weight_mat = {}
# default = [0,0,0,0,0.5]
# for i in id_to_tokens:
#     nested_dictionary = {}
#     for j in vocab:
#         nested_dictionary[j] = default
#     weight_mat[i]=nested_dictionary

# count = 0
# for i in id_to_tokens:
#     count+=1
#     if count%100==0:
#         print(count)
#     all_terms = id_to_tokens[i]
#     for j in all_terms:
#         weight_mat[i][j]=tf_idf(j,i)



In [ ]:
# f = open("weight_mat.txt","wb")
# pickle.dump(weight_mat,f)
# f.close()

In [3]:
f = open("weight_mat.txt","rb")
txt = f.read()
f.close()
weight_mat = pickle.loads(txt)

In [11]:
def relevant_by_tfidf(query,top_n_docs):
    query_token_list = process_query(query)
    
    score_binary = {}
    for i in id_to_tokens:
        score_binary[i]=0
    score_rawcount = score_binary.copy()
    score_termfrequency = score_binary.copy()
    score_lognorm = score_binary.copy()
    score_doublenorm = score_binary.copy()
    
    for j in query_token_list:
        for i in id_to_tokens:
            score_binary[i]+=weight_mat[i][j][0]
            score_rawcount[i]+=weight_mat[i][j][1]
            score_termfrequency[i]+=weight_mat[i][j][2]
            score_lognorm[i]+=weight_mat[i][j][3]
            score_doublenorm[i]+=weight_mat[i][j][4]

    sorted_binary = dict(sorted(score_binary.items(), key=lambda item: item[1],reverse=True))
    sorted_rawcount = dict(sorted(score_rawcount.items(), key=lambda item: item[1],reverse=True))
    sorted_termfrequency = dict(sorted(score_termfrequency.items(), key=lambda item: item[1],reverse=True))
    sorted_lognorm = dict(sorted(score_lognorm.items(), key=lambda item: item[1],reverse=True))
    sorted_doublenorm = dict(sorted(score_doublenorm.items(), key=lambda item: item[1],reverse=True))

    top_ids_binary = list(sorted_binary.keys())
    top_ids_rawcount = list(sorted_rawcount.keys())
    top_ids_termfrequency = list(sorted_termfrequency.keys())
    top_ids_lognorm = list(sorted_lognorm.keys())
    top_ids_doublenorm = list(sorted_doublenorm.keys())

    print("According to Binary Scheme")
    for i in range(top_n_docs):
        print(id_to_name[top_ids_binary[i]])
    print("")

    print("According to Raw Count Scheme")
    for i in range(top_n_docs):
        print(id_to_name[top_ids_rawcount[i]])
    print("")

    print("According to Term Frequency Scheme")
    for i in range(top_n_docs):
        print(id_to_name[top_ids_termfrequency[i]])
    print("")

    print("According to Log Normalisation Scheme")
    for i in range(top_n_docs):
        print(id_to_name[top_ids_lognorm[i]])
    print("")

    print("According to Double Normalisation Scheme")
    for i in range(top_n_docs):
        print(id_to_name[top_ids_doublenorm[i]])
    print("")
        
        
        
    

In [12]:
relevant_by_tfidf("helps with the digestion of rich",5)

According to Binary Scheme
1st_aid.txt
fiber.txt
urban.txt
ayurved.txt
fusion.sup

According to Raw Count Scheme
candy.txt
manners.txt
mlverb.hum
epi_merm.txt
melodram.hum

According to Term Frequency Scheme
1st_aid.txt
acronym.txt
oilgluts.hum
bredcake.des
beesherb.txt

According to Log Normalisation Scheme
candy.txt
manners.txt
1st_aid.txt
fiber.txt
mlverb.hum

According to Double Normalisation Scheme
1st_aid.txt
ayurved.txt
fusion.sup
fiber.txt
urban.txt



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=04d84d63-ff1d-4672-9f78-d034a2868658' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>